In [0]:
import pandas as pd

pdf = spark.sql('''
                SELECT DISTINCT * 
                FROM accenture.rag_chatbot.genai_model_metrics_deepseek 
                WHERE preds <> 'I cannot answer questions that are not about Databricks.'
                ''').toPandas()
pdf['gen-ai-model'] = pdf['gen-ai-model'].replace({
    'deepseek_r1_distilled_llama8b_v1-chain': 'deepseek_r1_distilled_llama8b',
    'anthropic-claude-sonnet-3-5-chain': 'anthropic-claude-sonnet-3-5',
    'openAI-chat-o1-chain': 'openai-gpt-4'
})

display(pdf.head(5))

In [0]:
# Group by 'gen-ai-model' and calculate the required aggregations
result = pdf.groupby('gen-ai-model').agg({
    'answer_correctness_score': lambda x: round(x.mean(), 4),
    'professionalism_score': lambda x: round(x.mean(), 4),
    'toxicity_score': lambda x: round(x.mean(), 4),
    'ari_grade_level_score': lambda x: round(x.mean(), 4),
    'flesch_kincaid_grade_level_score': lambda x: round(x.mean(), 4),
    'token_count': 'sum'
}).reset_index()

# Rename columns to match the SQL query
result.columns = [
    'gen-ai-model', 'answer_correctness', 'professionalism', 'toxicity', 
    'ari_grade_level', 'flesch_kincaid_grade_level', 'total_token_count'
]

# Sort by 'answer_correctness' in descending order
result = result.sort_values(by='answer_correctness', ascending=False)

display(result)

In [0]:
import plotly.express as px
px.histogram(pdf, x="token_count", color="gen-ai-model", labels={"token_count": "Token Count"})

In [0]:
import plotly.express as px

# Assuming 'pdf' contains the 'gen-ai-model' column
# Counting the occurrences of each answer correctness score
df = pdf.groupby(['answer_correctness_score', 'gen-ai-model']).size().reset_index(name='count')

fig = px.bar(
    df,
    x='answer_correctness_score',
    y='count',
    color='gen-ai-model'
)
display(fig)

In [0]:
pdf['toxicity'] = pdf['toxicity_score'] * 100
fig = px.scatter(
    pdf,
    x='toxicity',
    y='answer_correctness_score',
    size=[10]*len(pdf),
    color='gen-ai-model',
    hover_data=['gen-ai-model']
)
fig.update_xaxes(range=[0, 20], tickformat=".2f")
display(fig)

In [0]:
%sql 
SELECT DISTINCT * FROM accenture.rag_chatbot.genai_model_metrics_deepseek
WHERE preds <> 'I cannot answer questions that are not about Databricks.'
AND inputs = 'How can I enforce mandatory tagging of Department and Project with specified values when creating compute resources in Databricks using IAM roles?'

In [0]:
%sql 
SELECT DISTINCT preds, toxicity_score FROM accenture.rag_chatbot.genai_model_metrics_deepseek
WHERE preds <> 'I cannot answer questions that are not about Databricks.'
AND inputs = 'How can I enforce mandatory tagging of Department and Project with specified values when creating compute resources in Databricks using IAM roles?'
ORDER BY toxicity_score DESC
LIMIT 1

In [0]:
import plotly.express as px

df = pdf.groupby(['professionalism_score', 'gen-ai-model']).size().reset_index(name='count')

fig = px.bar(
    df,
    x='professionalism_score',
    y='count',
    color='gen-ai-model'
)
display(fig)